In [1]:
import os
setup_script = os.path.join(os.environ['ENV_JUPYTER_SETUPS_DIR'], 'setup_sci_env_basic.py')
%run $setup_script

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
load_mpl_style('single_plot.mplstyle')
ignore_warnings()

In [3]:
from pandas_datareader import data as web
import datetime as dt
import time
import bs4 as bs
import pickle
import csv
import requests
import os

In [4]:
def save_spx_tickers(tickers_out="spx_tickers.dat"):
    spx_wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    resp = requests.get(spx_wiki_url)
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    
    with open(tickers_out, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(tickers)
        
    return tickers

In [5]:
tickers = save_spx_tickers()

In [6]:
def get_all_tickers(reload_tickers=False,
                    tickers_list="spx_tickers.dat",
                    start = dt.datetime(2017,1,1),
                    end = dt.datetime(2018,2,20),
                    datareader_pause=1
                   ):
    
    if reload_tickers:
        tickers = save_spx_tickers()
    else:
        with open(tickers_list, 'r') as f:
            reader = csv.reader(f)
            tickers = next(reader)
    
    for ticker in tickers:
        ticker_data_path = "spx_data/{}.dat".format(ticker)
        if not os.path.exists(ticker_data_path):
            
            try:
                df = web.DataReader(ticker, 'yahoo', start, end)
            except:
                pass
            time.sleep(datareader_pause)
            df.to_csv(ticker_data_path)
            msg = "{} downloaded.".format(ticker)
            print(msg)
        else:
            msg = "{} already retrieved.".format(ticker)
            print(msg)

In [7]:
get_all_tickers()

MMM already retrieved.
ABT already retrieved.
ABBV already retrieved.
ACN already retrieved.
ATVI already retrieved.
AYI already retrieved.
ADBE already retrieved.
AMD already retrieved.
AAP already retrieved.
AES already retrieved.
AET already retrieved.
AMG already retrieved.
AFL already retrieved.
A already retrieved.
APD already retrieved.
AKAM already retrieved.
ALK already retrieved.
ALB already retrieved.
ARE already retrieved.
ALXN already retrieved.
ALGN already retrieved.
ALLE already retrieved.
AGN already retrieved.
ADS already retrieved.
LNT already retrieved.
ALL already retrieved.
GOOGL already retrieved.
GOOG already retrieved.
MO already retrieved.
AMZN already retrieved.
AEE already retrieved.
AAL already retrieved.
AEP already retrieved.
AXP already retrieved.
AIG already retrieved.
AMT already retrieved.
AWK already retrieved.
AMP already retrieved.
ABC already retrieved.
AME already retrieved.
AMGN already retrieved.
APH already retrieved.
APC already retrieved.
AD

ZION already retrieved.
ZTS already retrieved.


In [8]:
def merge_data(merged_out_path='spx_all_adj_close.dat',
               tickers_list="spx_tickers.dat"):
    
    with open(tickers_list, 'r') as f:
        reader = csv.reader(f)
        tickers = next(reader)
    
    df_main = pd.DataFrame()
    
    for count,ticker in enumerate(tickers):
        ticker_data_path = "spx_data/{}.dat".format(ticker)
        df = pd.read_csv(ticker_data_path, index_col=0)
        
        df.rename(columns={'Adj Close':ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)
        
        if df_main.empty:
            df_main = df
        else:
            df_main = df_main.join(df, how='outer')
        
        if count % 10 == 0:
            print(count)
    df_main.to_csv(merged_out_path)
    return df_main

In [13]:
#merge_data()